In [2]:
import numpy as np
import matplotlib.pyplot as plt

# Q1

In [43]:
def decomp1d(n, num_blocks, sub_mat_index):
    """_summary_

    Args:
        n (int): matrix lenght
        num_blocks (int): number of blocks to split into
        sub_mat_index (int): index of sub matrix row

    Returns:
        _type_: _description_
    """
    remainder = n % num_blocks
    base = n // num_blocks

    if (sub_mat_index < remainder):
        s =  sub_mat_index * (base+1)
        e = (sub_mat_index * (base+1)) + base+1
        return s, e  # start, end
    else:
        s = (remainder * (base+1) * remainder) + ((max(sub_mat_index-remainder, 0)) * base)
        e = (remainder * (base+1) * remainder) + ((max(sub_mat_index-remainder, 0)) * base) + base
        return s, e 
   
def decomp_matrix(A, block_rows, block_cols):
    a = [[0]*block_cols for i in range(block_rows)]
    m, n = A.shape
    
    for i in range(block_rows):
        s1, e1 = decomp1d(m, block_rows, i)
        for j in range(block_cols):
            s2, e2 = decomp1d(n, block_cols, j)
            a[i][j] = A[s1:e1, s2:e2]
    
    return a

def TSQR(A, num_rows, num_procs):
    m, n = A.shape
    W = decomp_matrix(A, block_rows=num_rows, block_cols=1)
    
    Q = 1
    R = 1
    
    for i in range(int(np.log2(num_procs))):
        size = num_procs//(i+1)
        
        Q_temp = [[0]*size for i in range(size)]
        R_temp = [0]*size
        
        for p in range(size):
            Qp, Rp = np.linalg.qr(W[p])
            Q_temp[p][p] = Qp
            #print(Qp.shape)
            R_temp[p] = Rp
        
        Q = Q * np.block(Q_temp)
        R = R_temp

    return Q, R


In [44]:
#A = np.random.rand(10, 15)
A = np.eye(10)
Q, R = np.linalg.qr(A)

print(Q)
print(R)

Q, R = TSQR(A, 4, 4)
print(Q)
print(R)

[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-0. -0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [-0. -0. -0.  1.  0.  0.  0.  0.  0.  0.]
 [-0. -0. -0. -0.  1.  0.  0.  0.  0.  0.]
 [-0. -0. -0. -0. -0.  1.  0.  0.  0.  0.]
 [-0. -0. -0. -0. -0. -0.  1.  0.  0.  0.]
 [-0. -0. -0. -0. -0. -0. -0.  1.  0.  0.]
 [-0. -0. -0. -0. -0. -0. -0. -0.  1.  0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -0.  1.]]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
(1, 3, 3)
(1, 3, 3)
(1, 0, 0)
(1, 0, 0)


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 3 and the array at index 1 has size 1

In [30]:
A = np.eye(10)
num = 3  
for i in range(num):
    for j in range(num):
        print(i, j)
        print(decomp_matrix(A, num)[i][j])

0 0
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
0 1
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
0 2
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
1 0
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
1 1
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
1 2
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
2 0
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
2 1
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
2 2
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
